In [104]:
import numpy as np 
import pandas as pd

In [105]:
calendar_df = pd.read_csv(r"E:\environments\wallmart_hackathon\dataset\m5-forecasting-accuracy\calendar.csv")
sales_validation_df = pd.read_csv(r"E:\environments\wallmart_hackathon\dataset\m5-forecasting-accuracy\sales_train_validation.csv")
sell_prices_df = pd.read_csv(r"E:\environments\wallmart_hackathon\dataset\m5-forecasting-accuracy\sell_prices.csv")

### Data Preprocessing

In [106]:
## melting last 120 days (form d_1789 to d_1913)
## lag = 28, rolling mean = 28

start_col = 1789
end_col = 1913

col = [f'd_{x}' for x in range(start_col, end_col + 1)]

columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'] + col

sub_validation_df = sales_validation_df[columns]

sales_long = sub_validation_df.melt(
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='sales'
)

print(sales_long.shape)
sales_long.head()

(3811250, 8)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1789,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1789,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1789,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1789,1
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1789,2


In [107]:
## merge with calendar

sales_long = sales_long.merge(calendar_df, how = 'left', on = 'd')

In [108]:
## convert date to datetime
sales_long['date'] = pd.to_datetime(sales_long['date'])

In [109]:
## merge with sales_prices
sales_long = sales_long.merge(sell_prices_df, how = 'left', on = ['store_id', 'item_id', 'wm_yr_wk'])

sales_long is the final dataset

In [110]:
sales_long.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1789,0,2015-12-22,11547,...,12,2015,NaN,NaN,NaN,NaN,0,0,0,8.26
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1789,0,2015-12-22,11547,...,12,2015,NaN,NaN,NaN,NaN,0,0,0,3.97
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1789,1,2015-12-22,11547,...,12,2015,NaN,NaN,NaN,NaN,0,0,0,2.97
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1789,1,2015-12-22,11547,...,12,2015,NaN,NaN,NaN,NaN,0,0,0,4.64
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1789,2,2015-12-22,11547,...,12,2015,NaN,NaN,NaN,NaN,0,0,0,2.88


In [111]:
sales_long = sales_long.sort_values(['id', 'date'])
sales_long.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1789,1,2015-12-22,11547,...,12,2015,NaN,NaN,NaN,NaN,0,0,0,2.24
32102,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1790,2,2015-12-23,11547,...,12,2015,NaN,NaN,NaN,NaN,0,0,0,2.24
62592,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1791,0,2015-12-24,11547,...,12,2015,NaN,NaN,NaN,NaN,0,0,0,2.24
93082,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1792,0,2015-12-25,11547,...,12,2015,Christmas,National,NaN,NaN,0,0,0,2.24
123572,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1793,1,2015-12-26,11548,...,12,2015,NaN,NaN,NaN,NaN,0,0,0,2.24


In [112]:
## create lag 28
sales_long['lag_28'] = (
    sales_long
    .groupby('id')['sales']
    .shift(28)
)
sales_long['lag_7'] = (
    sales_long
    .groupby('id')['sales']
    .shift(7)
)


In [113]:
sales_long['rolling_mean_28'] = (
    sales_long
    .groupby('id')['sales']
    .transform(
        lambda x: x.shift(1).rolling(window=28).mean()
    )
)



In [114]:
sales_long = sales_long.dropna(subset=['lag_28', 'rolling_mean_28'])
print(sales_long.shape)
sales_long.head()


(2957530, 25)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_28,lag_7,rolling_mean_28
855332,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1817,0,2016-01-19,11551,...,NaN,NaN,NaN,0,0,0,2.24,1.0,0.0,0.464286
885822,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1818,0,2016-01-20,11551,...,NaN,NaN,NaN,0,0,0,2.24,2.0,2.0,0.428571
916312,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1819,1,2016-01-21,11551,...,NaN,NaN,NaN,0,0,0,2.24,0.0,0.0,0.357143
946802,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1820,0,2016-01-22,11551,...,NaN,NaN,NaN,0,0,0,2.24,0.0,0.0,0.392857
977292,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1821,0,2016-01-23,11552,...,NaN,NaN,NaN,0,0,0,2.24,1.0,0.0,0.392857


In [115]:
sales_long['month'] = sales_long['date'].dt.month
sales_long['year'] = sales_long['date'].dt.year

sales_long['day_of_month'] = sales_long['date'].dt.day
sales_long['week_of_month'] = ((sales_long['day_of_month'] - 1) // 7) + 1

In [116]:
## target columns
# Reverse the time series so rolling looks "forward"
sales_long['sales_28_sum'] = (
    sales_long
    .iloc[::-1]                                 
    .groupby('id')['sales']
    .rolling(window=28, min_periods=28)
    .sum()
    .reset_index(level=0, drop=True)
    .iloc[::-1]                               
)

In [117]:
sales_long = sales_long.dropna(subset=['sales_28_sum'])

In [123]:
final_columns = ['item_id', 'dept_id', 'store_id', 'state_id', 'weekday', 'month', 'week_of_month', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_28', 'lag_7', 'rolling_mean_28', 'sales_28_sum' ]

final_df = sales_long[final_columns]
final_df

,item_id,dept_id,store_id,state_id,weekday,month,week_of_month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_28,lag_7,rolling_mean_28,sales_28_sum
855332,FOODS_1_001,FOODS_1,CA_1,CA,Tuesday,1,3,NaN,NaN,NaN,NaN,0,0,0,2.24,1.0,0.0,0.464286,23.0
885822,FOODS_1_001,FOODS_1,CA_1,CA,Wednesday,1,3,NaN,NaN,NaN,NaN,0,0,0,2.24,2.0,2.0,0.428571,23.0
916312,FOODS_1_001,FOODS_1,CA_1,CA,Thursday,1,3,NaN,NaN,NaN,NaN,0,0,0,2.24,0.0,0.0,0.357143,24.0
946802,FOODS_1_001,FOODS_1,CA_1,CA,Friday,1,4,NaN,NaN,NaN,NaN,0,0,0,2.24,0.0,0.0,0.392857,23.0
977292,FOODS_1_001,FOODS_1,CA_1,CA,Saturday,1,4,NaN,NaN,NaN,NaN,0,0,0,2.24,1.0,0.0,0.392857,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2864622,HOUSEHOLD_2_516,HOUSEHOLD_2,WI_3,WI,Thursday,3,4,Purim End,Religious,NaN,NaN,0,0,0,5.94,0.0,0.0,0.107143,0.0
2895112,HOUSEHOLD_2_516,HOUSEHOLD_2,WI_3,WI,Friday,3,4,NaN,NaN,NaN,NaN,0,0,0,5.94,0.0,0.0,0.107143,0.0
2925602,HOUSEHOLD_2_516,HOUSEHOLD_2,WI_3,WI,Saturday,3,4,NaN,NaN,NaN,NaN,0,0,0,5.94,0.0,0.0,0.107143,0.0
2956092,HOUSEHOLD_2_516,HOUSEHOLD_2,WI_3,WI,Sunday,3,4,Easter,Cultural,NaN,NaN,0,0,0,5.94,1.0,0.0,0.107143,0.0


In [129]:
final_df.nunique()

item_id            3049
dept_id               7
store_id             10
state_id              3
weekday               7
month                 3
week_of_month         5
event_name_1          8
event_type_1          4
event_name_2          0
event_type_2          0
snap_CA               2
snap_TX               2
snap_WI               2
sell_price          510
lag_28              159
lag_7               158
rolling_mean_28    1939
sales_28_sum       1977
dtype: int64

### Data Transformation

In [132]:
## train test split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


In [139]:
## Encoding the features
class PreProcessor:

    def __init__(self, dataframe):
        self.X = dataframe.drop('sales_28_sum', inplace = False, axis = 1)
        self.y = dataframe['sales_28_sum']
        self.encoders = {}
        self.scalers = None

    def get_train_test_split(self):
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size = 0.2, random_state = 42)
        return X_train, X_test, y_train, y_test
    
    def get_nan_value_imputing(self, X_train, X_test):
        event_cols = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
        for col in event_cols:
            X_train[col] = X_train[col].fillna('No_event')
            X_test[col] = X_test[col].fillna('No_event')
        return X_train, X_test
    
    def get_label_encoding(self, X_train, X_test):
        label_encoding_columns = ['item_id', 'dept_id', 'store_id', 'weekday', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
        for col in label_encoding_columns:
            label_encoder = LabelEncoder()
            X_train[col] = label_encoder.fit_transform(X_train[col])
            X_test[col] = X_test[col].map(lambda s: label_encoder.transform([s])[0] if str(s) in label_encoder.classes_ else -1)

            self.encoders[col] = label_encoder
        return X_train, X_test
    
    def get_scaled_data(self, X_train, X_test):
        numeric_cols = ['sell_price', 'lag_28', 'lag_7', 'rolling_mean_28']
        scaler = StandardScaler()
        X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
        X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])
        self.scaler = scaler
        return X_train, X_test
    
    def begin_preprocessing(self):
        X_tr, X_te, y_tr, y_te = self.get_train_test_split()
        X_tr, X_te = self.get_nan_value_imputing(X_tr, X_te)
        X_tr, X_te = self.get_label_encoding(X_tr, X_te)
        X_tr, X_te = self.get_scaled_data(X_tr, X_te)
        return X_tr, X_te, y_tr, y_te 

    

In [140]:
preprocessor = PreProcessor(final_df)
X_train, X_test, y_train, y_test = preprocessor.begin_preprocessing()

In [ ]:
class model_trainer:
    def __init__(self, X_train, y_train):
        pass